In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
def split_datetime(data, col="datetime"):
    # What columns are of type datetime?
    datetime_columns = data.select_dtypes(include='datetime64').columns
    
    for c in datetime_columns:
        print(f"Timezone for {c} is {data[c].dt.tz}")

    # Adding columns for date & time
    data['year']    = data[col].dt.year
    # data['quarter'] = data[col].dt.quarter
    data['month']   = data[col].dt.month
    data['week']    = data[col].dt.isocalendar().week
    data['hour']    = data[col].dt.hour 

    data['day_of_year']  = data[col].dt.day_of_year
    data['day_of_month'] = data[col].dt.day
    data['day_of_week']  = data[col].dt.day_of_week

    return data

In [ ]:
merged_df = pd.read_parquet('../data/merged_df.parquet')

In [ ]:
# encode categories to category datetype

merged_df['county'] = merged_df['county'].astype('category')
merged_df['product_type'] = merged_df['product_type'].astype('category')
merged_df['day_of_week'] = merged_df['day_of_week'].astype('category')


In [ ]:
# workday vs weekend
# estonian holidays
# increasing in the capacity
# aggregated weather of previous and/or future period
# aggregated prices of previous period
# target from previous year
# ...

In [ ]:
# copy df for modelling
model_df = merged_df

# model is not able to handle object type
model_df.drop('time_of_day', axis=1, inplace=True)

# split datetime into meaningful features of int types
model_df = split_datetime(model_df)

# model is not able to handle datetime
model_df = model_df.drop(model_df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, EET]']).columns, axis=1)

# drop na from target
model_df.dropna(subset=['target'], inplace=True)


In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(model_df.drop('target', axis=1), model_df['target'], test_size=0.3, random_state=0)

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
# first attempt gave us 50.75 mean absolute error

In [ ]:
model_df['data_block_id'].describe()

In [ ]:
# split of old data to train and newer one to test

Xy_train = model_df[model_df.data_block_id < 450]
X_train = Xy_train.drop('target', axis=1)
y_train = Xy_train.target

Xy_test = model_df[model_df.data_block_id >= 450]
X_test = Xy_test.drop('target', axis=1)
y_test = Xy_test.target

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
# divide by dates and use newer ones for validation

In [ ]:
xgb.plot_importance(bst)
plt.title('Feature Importance')
plt.show()

In [ ]:
# hours_ahead_forecast treated as important feature, probably smth to drop)

- visualisation
- split by date
- tweaking the parameters
- drop some features
- feature engineering
- overfitting with traditional train_test_split?
- try to models/ multiple_output/ other models

In [ ]:
# model_df2 = model_df.copy()
# model_df2.drop(['row_id', ])

split_datablock = 300

Xy_train = model_df[model_df.data_block_id < split_datablock]
X_train = Xy_train.drop('target', axis=1)
y_train = Xy_train.target

Xy_test = model_df[model_df.data_block_id >= split_datablock]
X_test = Xy_test.drop('target', axis=1)
y_test = Xy_test.target

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)

y_pred_test = bst.predict(X_test)
y_pred_train = bst.predict(X_train)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred_test))
print('Mean absolute error train', mean_absolute_error(y_train, y_pred_train))

In [ ]:
px.scatter(x=Xy_train.index, y=y_pred_train-y_train, color=Xy_train.month)

In [ ]:
px.scatter(data_frame=Xy_test, x=Xy_test.index, y=y_pred_test-y_test, color=Xy_test.month, hover_data='day_of_week')

In [ ]:
Xy_test['residual'] = y_pred_test-y_test

Xy_test.head()

In [ ]:
sns.heatmap(Xy_test.corr(numeric_only=True), annot=False, cmap='RdBu', center = 0)
plt.title('Correlation Heatmap')
plt.figure(
    figsize=(20, 20)
)
plt.show()

In [ ]:
target_column = 'residual'

# Exclude non-numeric columns
numeric_columns = Xy_test.select_dtypes(include=['number']).columns
numeric_df = Xy_test[numeric_columns]
numeric_df_cons = numeric_df[numeric_df['is_consumption'] == 1]

# Calculate the correlation matrix
correlation_matrix = numeric_df_cons.corr()

# Select correlations based on the threshold
threshold = 0.15
significant_correlations = correlation_matrix[(correlation_matrix[target_column] > threshold) | (correlation_matrix[target_column] < -threshold)][target_column]

# Plot a heatmap of the significant correlations with the target
plt.figure(figsize=(12, 8))
sns.heatmap(significant_correlations.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", cbar=False)
plt.title(f'Significant Correlations with {target_column}, CONSUM ONLY (Threshold: {threshold})')
plt.show()

In [ ]:
target_column = 'residual'

# Exclude non-numeric columns
numeric_columns = Xy_test.select_dtypes(include=['number']).columns
numeric_df = Xy_test[numeric_columns]
numeric_df_cons = numeric_df[numeric_df['is_consumption'] == 0]

# Calculate the correlation matrix
correlation_matrix = numeric_df_cons.corr()

# Select correlations based on the threshold
threshold = 0.15
significant_correlations = correlation_matrix[(correlation_matrix[target_column] > threshold) | (correlation_matrix[target_column] < -threshold)][target_column]

# Plot a heatmap of the significant correlations with the target
plt.figure(figsize=(12, 8))
sns.heatmap(significant_correlations.to_frame(), annot=True, cmap='coolwarm', fmt=".2f", cbar=False)
plt.title(f'Significant Correlations with {target_column}, PRODUCTION ONLY (Threshold: {threshold})')
plt.show()

- residuals are bigger at the summer time, we quess because production is happening at this time

- residuals on the test data have weekly pattern
- last two month predicted very poorly

- residuals are different depending on how we split our data 
- we see unexpalinable patterns in residuals
- residuals for consumption and production correlate with different features

- try residual analysis with traditional test_train_split

- tweak the model

In [ ]:
X_train, X_test, y_train,  y_test = train_test_split(model_df.drop('target', axis=1), model_df['target'], test_size=0.3, random_state=0)

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

y_pred_test = bst.predict(X_test)
y_pred_train = bst.predict(X_train)



- pretty much the same patterns with different train test splits


In [ ]:
model_df.columns

In [ ]:
drop_columns = [
    'target',
    'hours_ahead_forecast_weather',
    'row_id',
    'data_block_id', 
    'prediction_unit_id',
    'longitude_hist_weather',
    'longitude_forecast_weather',
    'latitude_hist_weather',
    'latitude_forecast_weather'
]

X_train, X_test, y_train,  y_test = train_test_split(model_df.drop(drop_columns, axis=1), model_df['target'], test_size=0.3, random_state=0)

bst = XGBRegressor(enable_categorical=True)
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)

# main optimisation metric
print('Mean absolute error test', mean_absolute_error(y_test, y_pred))
print('Mean absolute error train', mean_absolute_error(y_train, bst.predict(X_train)))

In [ ]:
px.scatter(x=X_train.index, y=y_pred_train-y_train, color=X_train.month)

In [ ]:
px.scatter(data_frame=X_test, x=X_test.index, y=y_pred_test-y_test, color=X_test.month, hover_data='day_of_week')

In [ ]:
xgb.plot_importance(bst)